In [66]:
import requests
from bs4 import BeautifulSoup
import pickle

ALL_CLASSES = 'http://nwmissouri.smartcatalogiq.com/2020-2021/Undergraduate-Catalog/Courses/CSIS-Computer-Science-Info-Systems'
PROGRAM_REQS = 'http://nwmissouri.smartcatalogiq.com/2020-2021/Undergraduate-Catalog/School-of-Computer-Science-and-Information-Systems/Computer-Science-and-Information-Systems-44/Computer-Science-Comprehensive-Major-6669-hours-BSNo-Minor-Required'
BASE_URL='http://nwmissouri.smartcatalogiq.com'

In [67]:
# get list of program requirements
reqpage = requests.get(PROGRAM_REQS)
reqsoup = BeautifulSoup(reqpage.text, 'html5lib')
reqlist = [x.text.split()[1] for x in reqsoup.findAll('a', {'class': 'sc-courselink'}) if x.text.startswith('CSIS')]
with open('csugrad.pkl', 'wb') as f:
    pickle.dump(reqlist, f)

In [69]:
# build prereq dependencies
classpage = requests.get(ALL_CLASSES)
classsoup = BeautifulSoup(classpage.text, 'html5lib')
classlist = classsoup.find('ul', {'class': 'sc-child-item-links'})

In [70]:
cnums = [x.text.split()[1] for x in classlist.findAll('a') if x.text.startswith('CSIS')]
cnames = {x.text.split()[1]: ' '.join(x.text.split()[2:]) for x in classlist.findAll('a') if x.text.startswith('CSIS')}
curls = {x.text.split()[1]: BASE_URL+x['href'] for x in classlist.findAll('a') if x.text.startswith('CSIS')}

In [46]:
dspage = requests.get(curls['44242'])

In [47]:
dssoup = BeautifulSoup(dspage.text)

In [51]:
prereqs = [x.text.split()[1] for x in dssoup.findAll('a', {'class':'sc-courselink'}) if x.text.startswith('CSIS')]

In [52]:
prereqs

['44241']

In [53]:
def pull_prereqs(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html5lib')
    return [x.text.split()[1] for x in soup.findAll('a', {'class':'sc-courselink'}) if x.text.startswith('CSIS')]

In [55]:
prereqs = {c: pull_prereqs(curls[c]) for c in curls}

In [56]:
with open ('prereqs.pkl', 'wb') as f:
    pickle.dump(prereqs, f)

In [57]:
prereqs

{'44100': [],
 '44101': [],
 '44125': [],
 '44130': [],
 '44141': ['44101', '44130'],
 '44144': ['44101', '44130'],
 '44149': [],
 '44155': [],
 '44181': ['44101', '44130'],
 '44200': [],
 '44241': ['44141'],
 '44242': ['44241'],
 '44249': ['44141'],
 '44296': ['44141'],
 '44300': [],
 '44312': ['44101', '44130'],
 '44317': ['44101', '44130'],
 '44320': ['44101', '44130'],
 '44330': ['44101', '44130'],
 '44334': ['44144'],
 '44335': ['44334', '44141', '44144'],
 '44336': ['44101', '44130'],
 '44337': ['44336'],
 '44340': ['44101', '44130'],
 '44343': ['44101', '44130'],
 '44345': ['44242'],
 '44346': ['44101', '44130'],
 '44349': ['44242', '44249'],
 '44351': ['44242'],
 '44356': ['44130', '44101', '44542', '44542'],
 '44359': ['44356'],
 '44382': ['44242', '44356'],
 '44386': ['44241', '44296', '44356'],
 '44400': [],
 '44410': [],
 '44411': ['44317'],
 '44413': ['44141'],
 '44418': ['44317', '44411', '44561'],
 '44425': [],
 '44432': [],
 '44440': ['44242'],
 '44443': ['44242'],
 '44

In [58]:
import json
with open('prereqs.json', 'w') as f:
    json.dump(prereqs, f)
with open('csugrad.json', 'w') as f:
    json.dump(reqlist, f)


In [59]:
bscs_prereqs = {c: prereqs[c] for c in reqlist}

In [61]:
with open('csonly_prereqs.json', 'w') as f:
    json.dump(bscs_prereqs, f)

In [64]:
import json
with open('csonly_prereqs.json', 'r') as f:
    bscs_prereqs = json.load(f)
    

In [71]:
import pickle
with open('cnames.pkl', 'wb') as f:
    pickle.dump(cnames, f)

In [87]:
nodes = [{'id': c, 'group': int(c[2])} for c in prereqs]

In [88]:
links = []
for c in prereqs:
    for pr in prereqs[c]:
        if pr[2] != '6':
            links.append({'source': pr, 'target': c, 'value': 1})

In [89]:
with open('csprereqs_arc.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links}, f)